In [15]:
import numpy as np
import polars as pl

import category_encoders as ce

import time
import math
import warnings

warnings.filterwarnings('ignore')

In [16]:
# load datasets
dataset = pl.read_csv('../data/competition/Train.csv', ignore_errors=True)
sbmssn = pl.read_csv('../data/competition/Test.csv', ignore_errors=True)

In [17]:
# preview dataset
dataset.head(n=5)

child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,child_observe_concentrated,child_observe_diligent,child_observe_interested,child_observe_total,child_gender,child_dob,child_zha,child_stunted,child_attends,child_attendance,child_languages,child_age_group,id_mn_best,prov_best,id_dc_best,dc_best,mn_best,ward_best,id_enumerator,id_facility,pqa_date,pqa_class_age,pqa_class_age_1,pqa_class_age_2,pqa_class_age_3,pqa_class_age_4,pqa_class_age_5,…,positionother,positionotherreason,sef_ind,language_match,elp_ind,gps_ind,pre_covid,ses_proxy,quintile_used,id_facility_n,id_ward_n,id_mn_n,id_dc_n,id_prov_n,language_assessment_w2,ses_cat,obs_lighting_1,obs_lighting_2,obs_lighting_3,obs_lighting_4,obs_lighting_5,obs_lighting_6,obs_lighting_8,obs_cooking_1,obs_cooking_2,obs_cooking_3,obs_cooking_4,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7,target
str,f64,str,f64,str,f64,str,str,f64,str,str,str,str,f64,str,str,f64,str,str,f64,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,…,str,str,str,f64,str,str,str,f64,str,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ID_SYSJ2FM0D""",2022.0,"""2022-02-03""",59.0,null,null,null,null,null,"""Sometimes""","""Sometimes""","""Sometimes""","""Sometimes""",4.0,"""Female""","""2017-02-06""",null,null,null,null,null,"""50-59 months""","""GT421""","""GAUTENG""","""DC42""","""SEDIBENG""","""EMFULENI""",39.0,20005.0,761.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""No""","""Yes""","""Post COVID""",2.0,"""Yes""",7.0,14.0,107.0,134.0,1051.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,51.5
"""ID_J5BTFOZR3""",2019.0,null,60.163933,null,null,null,"""1st year in th…",103.0,"""Sometimes""","""Almost never""","""Sometimes""","""Often""",4.0,"""Female""",null,-1.356791,"""Normal""",null,null,null,"""60-69 months""",null,"""KWAZULU-NATAL""","""DC22""","""UMGUNGUNDLOVU""",null,null,null,458.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""No""","""No""","""Pre-COVID""",4.0,"""Yes""",24.0,null,null,367.0,1832.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,55.869999
"""ID_R00SN7AUD""",2022.0,"""2022-03-11""",69.0,null,null,null,null,108.400002,"""Often""","""Often""","""Sometimes""","""Often""",7.0,"""Male""","""2016-05-24""",-1.250863,"""Normal""",null,null,null,"""60-69 months""","""CPT""","""WESTERN CAPE""","""CPT""","""CITY OF CAPE T…","""CITY OF CAPE T…",85.0,20001.0,925.0,null,null,null,null,null,null,null,…,null,null,"""No""",1.0,"""Yes""","""Yes""","""Post COVID""",1.0,"""No""",8.0,24.0,1448.0,1448.0,3214.0,null,"""R0-110""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,47.52
"""ID_BSSK60PAZ""",2021.0,"""2021-10-13""",53.0,"""2020-01-15""",20.0,"""No""","""1st year in th…",98.099998,"""Almost always""","""Almost always""","""Sometimes""","""Often""",9.0,"""Male""","""2017-05-08""",-1.830364,"""Normal""",null,null,null,"""50-59 months""","""WC025""","""WESTERN CAPE""","""DC2""","""CAPE WINELANDS…","""BREEDE VALLEY""",18.0,2689.0,308.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""Yes""","""Yes""","""Post COVID""",3.0,"""No""",4.0,22.0,76.0,629.0,3214.0,null,"""R291-750""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,58.599998
"""ID_IZTY6TC4D""",2021.0,"""2021-10-13""",57.0,"""2021-10-13""",0.0,null,"""2nd year in pr…",114.0,"""Almost always""","""Almost always""","""Almost always""","""Almost always""",12.0,"""Female""","""2016-12-19""",1.3292638,"""Normal""",null,null,null,"""50-59 months""","""KZN293""","""KWAZULU-NATAL""","""DC29""","""ILEMBE""","""NDWEDWE"

In [18]:
# init list of columns to drop - drop at end
drop_col = ['child_id']

In [19]:
# recode date columns
date_col = ['child_date', 'child_enrolment_date', 'child_dob', 'pqa_date', 'pra_date', 'pri_date', 'obs_date']

for cn in date_col: 
    dataset = dataset.with_columns(pl.col(cn).str.strptime(pl.Date, '%Y-%m-%d'))
    sbmssn = sbmssn.with_columns(pl.col(cn).str.strptime(pl.Date, '%Y-%m-%d'))

for cn in date_col: 
    dataset = dataset.with_columns([
        pl.col(cn).dt.year().alias(cn + '_year').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).dt.month().alias(cn + '_month').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).dt.quarter().alias(cn + '_quarter').cast(pl.Utf8).cast(pl.Categorical),
        pl.col(cn).dt.week().alias(cn + '_week').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).cast(pl.Int32)
    ])

    sbmssn = sbmssn.with_columns([
        pl.col(cn).dt.year().alias(cn + '_year').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).dt.month().alias(cn + '_month').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).dt.quarter().alias(cn + '_quarter').cast(pl.Utf8).cast(pl.Categorical),
        pl.col(cn).dt.week().alias(cn + '_week').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).cast(pl.Int32)
    ])

In [20]:
# geographic groups 
geo_cat = ['mn_best', 'prov_best', 'dc_best', 'ward_best']
geoid_cat = ['id_mn_best', 'id_prov', 'id_dc_best', 'id_ward']

dataset.select(pl.col(geo_cat)).dtypes

# do this later, don't drop bc not sure if those id_ represent the same info

[Utf8, Utf8, Utf8, Float64]

In [21]:
# coded as numerics we should treat them as categories (do later preprocessing with pipelines)
numcat_columns = ['id_enumerator', 'id_facility', 'id_team', 'ward_best', 'id_ward'] # FIXME Extension

for cn in numcat_columns: 
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Int64).cast(pl.Utf8).cast(pl.Categorical))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Int64).cast(pl.Utf8).cast(pl.Categorical))

In [22]:
# ordinal encoding for selected categories that may be of substantive interest

# child_age_group
dict = {'Younger than 50 months': 1, '50-59 months': 2, '60-69 months': 3, '70 Months or older': 4}
dataset = dataset.with_columns([
    pl.col('child_age_group').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
    ])
sbmssn = sbmssn.with_columns([
    pl.col('child_age_group').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
    ])

# 
process_cols = ['child_observe_attentive', 'child_observe_concentrated', 'child_observe_diligent', 'child_observe_interested']
dict = {'Almost never': 1, 'Sometimes': 2, 'Often': 3, 'Almost always': 4}

for cn in process_cols:
    dataset = dataset.with_columns([
        pl.col(cn).map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col(cn).map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
    ])

# 
process_cols = [s + '_ordinal' for s in process_cols] 
dataset = dataset.with_columns(pl.sum(process_cols).alias('child_observe_score'))
sbmssn = sbmssn.with_columns(pl.sum(process_cols).alias('child_observe_score'))

# child_years_in_programme
dict = {'1st year in the programme': 1, '2nd year in programme': 2, '3rd year in programme': 3}
dataset = dataset.with_columns([
    pl.col('child_years_in_programme').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
    ])
sbmssn = sbmssn.with_columns([
    pl.col('child_years_in_programme').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
    ])

# 
process_cols = ['pra_free_play', 'pra_free_play_outdoor']
dict = {'None': 1, '30 minutes or less': 2, 'Up to 1 hour': 3, 'Up to 2 hours': 4, 'More than 3 hours': 5}

for cn in process_cols:
    dataset = dataset.with_columns([
        pl.col(cn).map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col(cn).map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
    ])

#
dict = {'Seldom': 1, 'Sometime': 2, 'Often': 3}
dataset = dataset.with_columns([
    pl.col('pra_engaged').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
])

sbmssn = sbmssn.with_columns([
    pl.col('pra_engaged').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
])

#
dict = {'R0': 1, 'Less than R500 per month': 2, 'R500 – R749': 3, 'R750 – R999': 4, 'R1000 – R1249': 5, 
        'R1500 – R1999': 6, 'R2500 – R2999': 7, ' R3000 – R3499': 7, 'R3500 – R3999': 8, 'R4000 – R4449': 9, 
        'R 5000 – R5999': 10, 'More than R6000': 11}
dataset = dataset.with_columns([
    pl.col('pra_salary').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
])

sbmssn = sbmssn.with_columns([
    pl.col('pra_salary').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
])

#
dict = {'Other': 1, 'Below Grade 12/matric': 2, 'Certificate': 3, 'Matric/National Senior Certificate': 4, 'R1000 – R1249': 5, 
        'Diploma': 6, 'Undergraduate Degree': 7, ' Postgraduate degree': 7}
dataset = dataset.with_columns([
    pl.col('pra_education').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
])

sbmssn = sbmssn.with_columns([
    pl.col('pra_education').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
])

#
dict = {'Once a week': 1, 'Two times a week': 2, 'Three times a week': 3, 'Four times a week': 4, 'Five times a week': 5}
dataset = dataset.with_columns([
    pl.col('pri_attendance').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
])

sbmssn = sbmssn.with_columns([
    pl.col('pri_attendance').map_dict(dict, default=None, return_dtype=pl.Int64).suffix('_ordinal')
])

In [23]:
print(dataset['child_languages'].unique().to_list(), sep='\n')

['English + isiZulu +', 'Afrikaans', 'English + isiNdebele +', 'English + Afrikaans +', 'English + isiXhosa +', 'isiZulu', 'Setswana', 'Sesotho se Leboa (Sepedi)', 'isiZulu + isiXhosa +', 'English + Xitsonga +', 'isiXhosa + Xitsonga +', None, 'English + Sesotho se Leboa (Sepedi) +', 'Tshivenda', 'English + Setswana +', 'Sesotho', 'isiXhosa', 'Afrikaans + Setswana +', 'English + isiZulu + isiXhosa', 'English']


In [24]:
# extract child languages here
child_langs = ['Afrikaans', 'English', 'isiNdebele', 'isiXhosa',
               'isiZulu', 'Xitsonga', 'Setswana', 'Sesotho', 'Sepedi']

for ln in child_langs:
    dataset = dataset.with_columns([
        pl.col('child_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'child_languages_{ln}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('child_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'child_languages_{ln}')
    ])

drop_col.append('child_languages')

In [25]:
# FIXME count languages a child speaks
dataset = dataset.with_columns([
    pl.sum(
        pl.col('^child_languages.*$').
        cast(pl.Utf8).
        str.contains('Yes').
        cast(pl.Int32)
    ).alias('child_languages_count')
])

sbmssn = sbmssn.with_columns([
    pl.sum(
        pl.col('^child_languages.*$').
        cast(pl.Utf8).
        str.contains('Yes').
        cast(pl.Int32)
    ).alias('child_languages_count')
])

In [26]:
print(dataset['pri_languages'].unique().to_list(), sep='\n')

['English + Afrikaans +', 'Sesotho', 'isiXhosa', 'English', 'English + isiZulu +', 'isiZulu', None, 'Afrikaans', 'English + Afrikaans + isiXhosa', 'Setswana', 'Afrikaans + isiZulu +', 'English + isiXhosa +']


In [27]:
# extract pri_languages here
pri_langs = ['Afrikaans', 'English', 'isiXhosa', 'isiZulu', 'Setswana', 'Sesotho']

for ln in pri_langs:
    dataset = dataset.with_columns([
        pl.col('pri_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_languages_{ln}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_languages_{ln}')
    ])

drop_col.append('pri_languages')

# Convert this to categorical
dataset = dataset.with_columns([
    pl.col('language_match').map_dict({0: 'No', 1: 'Yes'}, default=None).cast(pl.Categorical)
])

sbmssn = sbmssn.with_columns([
    pl.col('language_match').map_dict({0: 'No', 1: 'Yes'}, default=None).cast(pl.Categorical)
])

In [28]:
# fix the gps column
dataset = dataset.with_columns(
    pl.col('gps').str.split(' ').\
    arr.to_struct(n_field_strategy='max_width').\
    struct.rename_fields(['gps_lat', 'gps_lon'])).\
    unnest('gps')

sbmssn = sbmssn.with_columns(
    pl.col('gps').str.split(' ').\
    arr.to_struct(n_field_strategy='max_width').\
    struct.rename_fields(['gps_lat', 'gps_lon'])).\
    unnest('gps')

# cast to numeric
for cn in ['gps_lat', 'gps_lon']:
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Float64))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Float64))

drop_col.append('gps_ind')

In [29]:
# some strings should be encoded as NA (probably not exhaustive)
dataset = dataset.with_columns([
    pl.when(pl.col(pl.Utf8).str.contains('Do Not Know|Missing|Don\'t know|Refuse|DON\'T KNOW|DON\'T KNOW.|DON\'T KNOW THE REASON FOR CONDITIONAL'))
    .then(None)
    .otherwise(pl.col(pl.Utf8))
    .keep_name()
])

sbmssn = sbmssn.with_columns([
    pl.when(pl.col(pl.Utf8).str.contains('Do Not Know|Missing|Don\'t know|Refuse|DON\'T KNOW|DON\'T KNOW.|DON\'T KNOW THE REASON FOR CONDITIONAL'))
    .then(None)
    .otherwise(pl.col(pl.Utf8))
    .keep_name()
])

In [30]:
# check where dataset and sbmssn diasgree on data types
for cn in dataset.select(pl.exclude('target')).columns:
    if dataset[cn].dtype != sbmssn[cn].dtype:
        print(f'Column {cn} has different data types between the two data frames.')

Column pri_network_type has different data types between the two data frames.
Column pri_staff_changes_reasons has different data types between the two data frames.
Column pri_food_donor has different data types between the two data frames.


In [31]:
# fix those columns
for answer in [1, 2, 3, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_network_type').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_network_type_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_network_type').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_network_type_{answer}')
    ])

for answer in [1, 2, 3, 4, 5, 6, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_staff_changes_reasons').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_staff_changes_reasons_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_staff_changes_reasons').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_staff_changes_reasons_{answer}')
    ])

for answer in [1, 2, 3, 4, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_food_donor').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_food_donor_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_food_donor').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_food_donor_{answer}')
    ])

# drop the splitted columns
drop_col.extend(['pri_network_type', 'pri_staff_changes_reasons', 'pri_food_donor'])

In [32]:
# categorical encoding of some other survey responses
for answer in [1, 2, 3, 4, 5, 97]:
    dataset = dataset.with_columns([
        pl.col('health').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'health_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('health').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'health_{answer}')
    ])

# drop the splitted columns
drop_col.append('health')

In [33]:
# categories that are encoded yes / no convert to 1 / 0 
# FIXME


In [34]:
# get a list of the column names ordered by the least NAs they have
# the more missing, the less it is going to contribute to the model
threshold_missings = 6000 # FIXME
exclude_by_na = dataset.to_pandas().isna().sum().sort_values(ascending=True).where(lambda x : x > threshold_missings).dropna()
drop_col.extend(exclude_by_na.index.to_list())

In [35]:
drop_col.extend(['obs_materials', 'obs_handwashing', 'obs_area', 'obs_toilet',
    'obs_equipment', 'obs_safety', 'obs_hazard', 'pqa_class_age',
    'pra_groupings', 'pra_plans', 'obs_access_disability', 'pra_cohort',
    'pra_plan_4yrs', 'pra_plan_5yrs', 'pra_plan_4yrsother', 'pra_plan_5yrsother',
    'pra_qualification', 'pra_ncf_trainer', 'pra_training', 'pri_language',
    'pri_meal', 'pri_money',  'pri_funding_salary', 'pri_expenseother',
    'pri_clinic_travel', 'pri_covid_awareness', 'pri_covid_precautions',
    'pri_food_type', 'pri_records', 'pri_support_provider', 
    'pri_qualification', 'pri_email_network_forum', 
    'pri_reason_register_year'])

In [36]:
# drop columns
dataset = dataset.select(pl.all().exclude(drop_col))
sbmssn = sbmssn.select(pl.all().exclude(list(set(drop_col) - set(['child_id']))))

In [37]:
# other strings to categories
for cn in dataset.select(pl.col(pl.Utf8)).columns: 
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Categorical))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Categorical))

In [38]:
# summary stats
dataset.shape
types = dataset.dtypes
{x:types.count(x) for x in types}

{Float64: 68, Int32: 3, Categorical: 204, Int64: 15}

In [39]:
# save as parquet for use in tuning and prediction
dataset.write_parquet("../data/clean/dataset.parquet")
sbmssn.write_parquet("../data/clean/sbmssn.parquet")